# Vanilla GAN with Korean

### Lab Overview

img

### Prerequisite

In [69]:
import torch 
import os

# mps setting
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"{device} is available.")

mps is available.


In [25]:
# save model
model_dir = os.path.join('result/2/model')
os.makedirs(model_dir, exist_ok=True)

image_dir = os.path.join('result/2/sample')
os.makedirs(image_dir, exist_ok=True)

loss_dir = os.path.join('result/2/loss')
os.makedirs(loss_dir, exist_ok=True)

### Import Libraries

In [70]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchsummary import summary
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import numpy as np

### Generate Korean Font Image

In [26]:
import glob, io
from PIL import Image, ImageFont, ImageDraw

TXT_FILE = 'data/characters/50characters.txt'
FONTS_DIR = 'data/fonts'
IMAGE_DIR = 'data/target/2'

if not os.path.exists(IMAGE_DIR):
    os.makedirs(os.path.join(IMAGE_DIR))

In [49]:
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28

list_labels = []
with open(TXT_FILE, 'r', encoding='utf-8') as fr:
    for line in fr:
        list_labels.append(line.strip())

with io.open(TXT_FILE, 'r', encoding='utf-8') as f:
    labels = f.read().splitlines()

# Get a list of the fonts.
fonts = sorted(glob.glob(os.path.join(FONTS_DIR, '*.ttf')))
for f in fonts:
    filename = os.path.basename(f)
    filename_without_extension = os.path.splitext(filename)[0]
    print(filename_without_extension)

# Initialize numbers
total_count = 0
prev_count = 0
font_count = 1
char_no = 0
    
# Total number of font files is 
print('total number of fonts are ', len(fonts))

for character in labels:
    char_no += 1
        
    for font in fonts:
        total_count += 1

        image = Image.new('RGB', (IMAGE_WIDTH,IMAGE_HEIGHT), (0, 0, 0))
        w, h = image.size
                
        drawing = ImageDraw.Draw(image)
        font = ImageFont.truetype(font, 15)

        box = None
        new_box = drawing.textbbox((0, 0), character, font)
                
        new_w = new_box[2] - new_box[0]
        new_h = new_box[3] - new_box[1]
                
        box = new_box
        w = new_w
        h = new_h
                
        x = (IMAGE_WIDTH - w)//2 - box[0]
        y = (IMAGE_HEIGHT - h)//2 - box[1]

        drawing.text((x,y), character, fill=(255,255,255), font=font) 
        file_string = '{}.png'.format(hex(ord(character))[2:].upper())
        file_path = os.path.join(IMAGE_DIR, file_string)
        image.save(file_path, 'PNG')
        font_count += 1

    font_count = 1
char_no = 0
            
print('Finished generating {} images.'.format(total_count))

GothicA1-Regular
total number of fonts are  1
Finished generating 50 images.


### Datasets

In [27]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, file) for file in os.listdir(root_dir) if file.endswith('.png')]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

In [28]:
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = CustomDataset(root_dir='data/target/2', transform=transform)

batch_size = 64
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


### Hyperparameters

In [29]:
NOISE = 100
INPUT_SIZE = 28*28
BATCH_SIZE = 64
EPOCHS = 50

### Generator

In [30]:
class Generator(nn.Module):
    def __init__(self, noise):
        super(Generator, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(noise, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 28*28),
            nn.Tanh()
        )
    
    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), 1, 28, 28)
        return img

### Discriminator

In [31]:
class Discriminator(nn.Module):
    def __init__(self, input_size):
        super(Discriminator, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.model(x)
        return x

### Configurate Model

In [67]:
generator = Generator(NOISE).to(device)
print(f"Generate is using {device}.")
print(generator)

Generate is using mps.
Generator(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): LeakyReLU(negative_slope=0.2)
    (6): Linear(in_features=1024, out_features=784, bias=True)
    (7): Tanh()
  )
)


In [68]:
discriminator = Discriminator(INPUT_SIZE).to(device)
print(f"Discriminator is using {device}")
print(discriminator)

Discriminator is using mps
Discriminator(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=256, out_features=128, bias=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Dropout(p=0.3, inplace=False)
    (12): Linear(in_features=128, out_features=1, bias=True)
    (13): Sigmoid()
  )
)


### Loss and Optimizer

In [13]:
criterion = nn.BCELoss()
optimizer_discriminator = optim.Adam(discriminator.parameters(), lr=1e-4)
optimizer_generator = optim.Adam(generator.parameters(), lr=1e-4)

In [14]:
for param in discriminator.parameters():
    param.requires_grad = False

In [15]:
gan_input = torch.randn(BATCH_SIZE, NOISE).to(device)
x = generator(gan_input)
output = discriminator(x)

### Visualize

In [16]:
def visualize_training(epoch, d_losses, g_losses, output_dir=image_dir, loss_dir=loss_dir):

    plt.figure(figsize=(8, 4))
    plt.plot(d_losses, label='Discriminator Loss')
    plt.plot(g_losses, label='Generatror Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    plt.close()
    
    print('epoch: {}, Discriminator Loss: {}, Generator Loss: {}'.format(epoch, np.asarray(d_losses).mean(), np.asarray(g_losses).mean()))
    
    #Visualize after creating sample data
    noise = torch.randn(24, NOISE).to(device)
    generated_images = generator(noise).cpu().detach().numpy()
    generated_images = generated_images.reshape(-1, 28, 28)
    
    plt.figure(figsize=(8, 4))
    for i in range(generated_images.shape[0]):
        plt.subplot(4, 6, i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'generated_images_epoch_{epoch}.png'))
    plt.close()
    plt.show()


### Save

In [17]:
def save_training(epoch, d_losses, g_losses, output_dir=image_dir, loss_dir=loss_dir):
    plt.figure(figsize=(8, 4))
    plt.plot(d_losses, label='Discriminator Loss')
    plt.plot(g_losses, label='Generatror Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'epoch: {epoch}, Discriminator Loss: {np.asarray(d_losses).mean():.4f}, Generator Loss: {np.asarray(g_losses).mean():.4f}')
    plt.savefig(os.path.join(loss_dir, f'generated_images_epoch_{epoch}.png'))
    plt.close()
        
    #Visualize after creating sample data
    noise = torch.randn(24, NOISE).to(device)
    generated_images = generator(noise).cpu().detach().numpy()
    generated_images = generated_images.reshape(-1, 28, 28)
    
    plt.figure(figsize=(8, 4))
    for i in range(generated_images.shape[0]):
        plt.subplot(4, 6, i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'generated_images_epoch_{epoch}.png'))
    plt.close()


### Train

In [18]:
d_losses = []
g_losses = []

for epoch in range(1, EPOCHS + 1):
    for i, real_images in enumerate(train_loader):
        batch_size = real_images.size(0)
        real_images = real_images.to(device)

        # Real and fake labels
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Train Discriminator
        discriminator.train()
        optimizer_discriminator.zero_grad()

        outputs = discriminator(real_images)
        d_loss_real = criterion(outputs, real_labels)
        d_loss_real.backward()

        z = torch.randn(batch_size, NOISE).to(device)
        fake_images = generator(z)
        outputs = discriminator(fake_images.detach())
        d_loss_fake = criterion(outputs, fake_labels)
        d_loss_fake.backward()

        optimizer_discriminator.step()

        # Train Generator
        generator.train()
        optimizer_generator.zero_grad()

        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()

        optimizer_generator.step()

        # Record losses
        d_losses.append(d_loss_real.item() + d_loss_fake.item())
        g_losses.append(g_loss.item())

    # Visualize and save models periodically
    if epoch == 1 or epoch % 10 == 0:
        visualize_training(epoch, d_losses, g_losses)
        torch.save(generator.state_dict(), os.path.join(model_dir, f'generator.pth'))
        torch.save(discriminator.state_dict(), os.path.join(model_dir, f'discriminator.pth'))


RuntimeError: linear(): input and weight.T shapes cannot be multiplied (50x2352 and 784x1024)